In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
import pygmt
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, regularizers,optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler
import keras
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# ship depth
data = pygmt.select(region="-15.5/5.5/-4.5/4.5",data="data/guinea.txt")
control, check = train_test_split(data, train_size=0.8, test_size=0.2, random_state=17)

# short G
rou=1
d=-8000
control_short=(control.values[:,2]-d)*2*3.1415*6.67259*(10**-8)*rou*100000 # short wavelength gravity ano
control_short = np.reshape(control_short, (len(control_short), 1))
short = np.concatenate([control.values[:,0:2],control_short],axis=1)
# short = pygmt.xyz2grd(data=short,spacing="15s",region="142.5/147.5/22.5/27.5")
pygmt.surface(region="-15.5/5.5/-4.5/4.5", spacing="1m",outgrid="short.grd",data=short)

In [ ]:
# ############### train data #######################################################
control_lon = np.reshape(control.values[:,0],(-1,1))
control_lat = np.reshape(control.values[:,1],(-1,1))
control_depth = np.reshape(control.values[:,2],(-1,1))
# free
# vgg
# nvd
# evd

# ################# normalization ################
# ############### scaler ######################
scalerlon = MinMaxScaler().fit(control_lon)
scalerlat = MinMaxScaler().fit(control_lat)
scalerdepth = MinMaxScaler().fit(control_depth)
# scalershort G
scalershort = MinMaxScaler().fit(control_short)  #short gravity
# scalerfree
# scalervgg
# scalernvd
# scalerevd
# ############## nomalizate ###############
lon_train = scalerlon.fit_transform(control_lon)
lat_train = scalerlat.fit_transform(control_lat)
depth_train = scalerdepth.fit_transform(control_depth)
# shortG
short_train = scalershort.fit_transform(control_short)
# free
# vgg

# nvd
# evd

# x_trian y_train
x_train = np.concatenate((lon_train,lat_train,short_train),axis=1)
y_train = depth_train
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train)


# ###################################################### test data #################################
check_short = pygmt.grdtrack(points=check.values,grid="short.grd",no_skip=True)
check_short = np.reshape(check_short.values[:,3],(-1,1))
check_lon = np.reshape(check.values[:,0],(-1,1))
check_lat = np.reshape(check.values[:,1],(-1,1))
check_depth = np.reshape(check.values[:,2],(-1,1))
# free
# vgg
# nvd
# evd

# ################# normalization ################

lon_test = scalerlon.fit_transform(check_lon)
lat_test= scalerlat.fit_transform(check_lat)
depth_test = scalerdepth.fit_transform(check_depth)
# shortG
short_test = scalershort.fit_transform(check_short)
# free
# vgg
# nvd
# evd

# x_test y_test
x_test = np.concatenate((lon_test,lat_test,short_test),axis=1)
y_test = depth_test

In [ ]:
clf = RandomForestRegressor (n_estimators=100, random_state=60)
clf.fit(x_train, y_train)

In [ ]:
score = clf.score(x_test, y_test)
score

In [ ]:
pre = clf.predict(x_test)
pre.shape
pre_raw = scalerdepth.inverse_transform(np.reshape(pre,(-1,1)))
check_depth = scalerdepth.inverse_transform(y_test)
m = np.mean(check_depth-pre_raw)
std = np.std(check_depth-pre_raw)
print(std,m)

In [ ]:
free = pygmt.grdcut(grid="D:/MAwork/zhonglifanyanshuishen/grav2depth/GeoModel//Free_Air_Gravity_Anomalies/Free_Air_Gravity_Anomalies.nc", region="-15.5/5.5/-4.5/4.5")
free.to_netcdf("free3.grd")
free = pygmt.grd2xyz(grid=free, region="-15.5/5.5/-4.5/4.5")
free_1m = pygmt.surface(region="-15.5/5.5/-4.5/4.5", spacing="1m",data=free)

In [ ]:
control_lon = np.reshape(control.values[:,0],(-1,1))
control_lat = np.reshape(control.values[:,1],(-1,1))
control_depth = np.reshape(control.values[:,2],(-1,1))

control_free = pygmt.grdtrack(points=control.values,grid="free3.grd",no_skip=True)
control_free = np.reshape(control_free.values[:,3], (len(control_free), 1))
control_long = control_free-control_short
long = np.concatenate([control.values[:,0:2],control_long],axis=1)
# long
long = pygmt.surface(region="-15.5/5.5/-4.5/4.5", spacing="1m",data=long)
short = free_1m - long

In [ ]:
free_short = pygmt.grd2xyz(grid=short, region="-15.5/5.5/-4.5/4.5")
fs_short = np.reshape(free_short.values[:,2],(-1,1))

fs_lon = np.reshape(free_short.values[:,0],(-1,1))
fs_lat = np.reshape(free_short.values[:,1],(-1,1))

In [ ]:
lon_pre = scalerlon.fit_transform(fs_lon)
lat_pre= scalerlat.fit_transform(fs_lat)

# shortG
short_pre = scalershort.fit_transform(fs_short)
x_pre = np.concatenate((lon_pre,lat_pre,short_pre),axis=1)

In [ ]:
pre_all = clf.predict(x_pre)
pre_raw_all = scalerdepth.inverse_transform(np.reshape(pre_all,(-1,1)))
fs_lon = scalerlon.inverse_transform(np.reshape(x_pre[:,0],(-1,1)))
fs_lat = scalerlat.inverse_transform(np.reshape(x_pre[:,1],(-1,1)))

In [ ]:
pre_sg = np.concatenate((fs_lon,fs_lat,pre_raw_all),axis=1)

In [ ]:
pygmt.surface(region="142.6/147.3/23/27", spacing="1m",outgrid="depth_forest.grd",data=pre_sg)